# Custom Network (Broken)

Gradients keep exploading and it wouldn't use the loss function I built for it correctly

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import pprint
from pickle import dump

In [ ]:
songs = pd.read_csv("song_list5.csv")

In [ ]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms"]].to_numpy()

In [ ]:
scaler = StandardScaler()

In [ ]:
x_train = scaler.fit_transform(features)

In [ ]:
dump(scaler, open('scaler2.pkl', 'wb'))

In [ ]:
class NeuralNetwork: 
    
    def __init__(self):
        # Setup Arch
        self.inputs = 12 # Input Layer

        self.encode1 = 512
        self.encode2 = 256 # 3 Dense Layers
        self.encode3 = 128

        self.latent = 2 # Latent Space Dense Layer

        self.decode1 = 128
        self.decode2 = 256 # 3 Dense Layers
        self.decode3 = 512

        self.outputs = 12 # Output Layer
        
        # Initialize Weights
        self.weights1 = np.random.randn(self.inputs, self.encode1)
        self.weights2 = np.random.randn(self.encode1, self.encode2)
        self.weights3 = np.random.randn(self.encode2, self.encode3)
        self.weights4 = np.random.randn(self.encode3, self.latent)
        self.weights5 = np.random.randn(self.latent, self.decode1)
        self.weights6 = np.random.randn(self.decode1, self.decode2)
        self.weights7 = np.random.randn(self.decode2, self.decode3)
        self.weights8 = np.random.randn(self.decode3, self.outputs)

    def relu(self, X):
        return np.maximum(0,X)
    def reluPrime(self, X):
        X[X<=0] = 0
        X[X>0] = 1

    def predict(self, X):
        """
        Calculate the NN inference using feed forward.
        """
        # ===================Encoder========================
        # Inputs to Layer 1
        self.hidden_sum1 = np.dot(X, self.weights1)
        self.activated_hidden1 = self.relu(self.hidden_sum1)
        # Layer 1 to Layer 2
        self.hidden_sum2 = np.dot(self.activated_hidden1, self.weights2)
        self.activated_hidden2 = self.relu(self.hidden_sum2)
        # Layer 2 to Layer 3
        self.hidden_sum3 = np.dot(self.activated_hidden2, self.weights3)
        self.activated_hidden3 = self.relu(self.hidden_sum3)
        # ================Latent Space======================
        # Layer 3 to Latent
        self.hidden_sum4 = np.dot(self.activated_hidden3, self.weights4)
        # ===================Decoder========================
        # Latent to Layer 1
        self.hidden_sum5 = np.dot(self.hidden_sum4, self.weights5)
        self.activated_hidden4 = self.relu(self.hidden_sum5)
        # Layer 1 to Layer 2
        self.hidden_sum6 = np.dot(self.activated_hidden4, self.weights6)
        self.activated_hidden5 = self.relu(self.hidden_sum6)
        # Layer 2 to Layer 3
        self.hidden_sum7 = np.dot(self.activated_hidden5, self.weights7)
        self.activated_hidden6 = self.relu(self.hidden_sum7)
        # Layer 3 to Output
        self.hidden_sum8 = np.dot(self.activated_hidden6, self.weights8)

        return self.hidden_sum8

    def backward(self, X,o):
        """
        Back prop thru the network
        """
        
        self.o_error = X-o
        self.o_delta = self.o_error * self.reluPrime(self.hidden_sum8)


In [ ]:
    def backward(self, X,y,o):
        """
        Back prop thru the network
        """
        
        self.o_error = y - o # Y - Output to get error
        
        # Apply derivative of sigmoid to error
        self.o_delta = self.o_error * self.sigmoidPrime(self.output_sum) # Error * Weights(with activation)
        
        # z2 error: how much were our output layer weights off
        self.z2_error = self.o_delta.dot(self.weights2.T) # Get the error of the Next layer
        
        # z2 delta: how much were the weights off?
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.hidden_sum) # error * Weights(with activation)

        self.weights1 += X.T.dot(self.z2_delta) #Adjust first set (input => hidden) weights
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) #adjust second set (hidden => output) weights
        
    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [ ]:
def relu(X):
    return np.maximum(0,X)
def reluPrime(X):
    x = X.copy()
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [ ]:
test_weights1 = np.random.randn(12, 512)
test_weights2 = np.random.randn(512, 256)
test_weights3 = np.random.randn(256, 128)
test_weights4 = np.random.randn(128, 2)
test_weights5 = np.random.randn(2, 128)
test_weights6 = np.random.randn(128, 256)
test_weights7 = np.random.randn(256, 512)
test_weights8 = np.random.randn(512, 12)

In [ ]:
x_test = np.atleast_2d(x_train)

In [ ]:
# Testing the prediction function code to ensure it is working

# ===================Encoder========================
# Input to Layer 1
test_hidden_sum1 = np.dot(x_test, test_weights1)
test_activated_hidden1 = relu(test_hidden_sum1)
# Layer 1 to Layer 2
test_hidden_sum2 = np.dot(test_activated_hidden1, test_weights2)
test_activated_hidden2 = relu(test_hidden_sum2)
# Layer 2 to Layer 3
test_hidden_sum3 = np.dot(test_activated_hidden2, test_weights3)
test_activated_hidden3 = relu(test_hidden_sum3)
# ================Latent Space======================
# Layer 3 to Latent
test_hidden_sum4 = np.dot(test_activated_hidden3, test_weights4)
# ===================Decoder========================
# Latent to Layer 1
test_hidden_sum5 = np.dot(test_hidden_sum4, test_weights5)
test_activated_hidden4 = relu(test_hidden_sum5)
# Layer 1 to Layer 2
test_hidden_sum6 = np.dot(test_activated_hidden4, test_weights6)
test_activated_hidden5 = relu(test_hidden_sum6)
# Layer 2 to Layer 3
test_hidden_sum7 = np.dot(test_activated_hidden5, test_weights7)
test_activated_hidden6 = relu(test_hidden_sum7)
# Layer 3 to Output
test_hidden_sum8 = np.dot(test_activated_hidden6, test_weights8)

In [ ]:
# Testing the backprop function

learn_rate = .00000001

# Error of the Output Layer
output_error = x_test - test_hidden_sum8
delta_output = output_error * test_hidden_sum8
# Error of the Decoder Layer 3
decode_3_error = delta_output.dot(test_weights8.T)
delta_decode_3 = decode_3_error * reluPrime(test_hidden_sum7)
# Error of the Decoder Layer 2
decode_2_error = delta_decode_3.dot(test_weights7.T)
delta_decode_2 = decode_2_error * reluPrime(test_hidden_sum6)
# Error of the Decoder Layer 1
decode_1_error = delta_decode_2.dot(test_weights6.T)
delta_decode_1 = decode_1_error * reluPrime(test_hidden_sum5)
# Error of the Latent Layer
latent_error = decode_1_error.dot(test_weights5.T)
delta_latent = latent_error * test_hidden_sum4
# Error of the Encoder Layer 3
encode_3_error = latent_error.dot(test_weights4.T)
delta_encode_3 = encode_3_error * reluPrime(test_hidden_sum3)
# Error of the Encoder Layer 2
encode_2_error = encode_3_error.dot(test_weights3.T)
delta_encode_2 = encode_2_error * reluPrime(test_hidden_sum2)
# Error of the Encoder Layer 1
encode_1_error = encode_2_error.dot(test_weights2.T)
delta_encode_1 = encode_1_error * reluPrime(test_hidden_sum1)

test_weights1 += x_test.T.dot(delta_encode_1) * learn_rate
test_weights2 += test_activated_hidden1.T.dot(delta_encode_2) * learn_rate
test_weights3 += test_activated_hidden2.T.dot(delta_encode_3) * learn_rate
test_weights4 += test_activated_hidden3.T.dot(delta_latent) * learn_rate
test_weights5 += test_hidden_sum4.T.dot(delta_decode_1) * learn_rate
test_weights6 += test_activated_hidden4.T.dot(delta_decode_2) * learn_rate
test_weights7 += test_activated_hidden5.T.dot(delta_decode_3) * learn_rate
test_weights8 += test_activated_hidden6.T.dot(delta_output) * learn_rate

print(abs(output_error).mean())
print(abs(output_error.mean()))

# VAE

In [1]:
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
import pprint
from pickle import dump

Using TensorFlow backend.


In [2]:
songs = pd.read_csv("song_list5.csv")

In [3]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms"]].to_numpy()

In [4]:
scaler = StandardScaler()

In [5]:
x_train = scaler.fit_transform(features)

In [6]:
dump(scaler, open('scaler2.pkl', 'wb'))

In [7]:
x_train[0]

array([-0.47695143, -1.43616823,  1.03467011, -1.32564479,  0.70711739,
       -0.34028656,  1.91669133, -0.14321669,  3.87500921, -1.70714343,
       -0.17066299,  0.03404049])

In [8]:
# reparameterization method for lambda layer
# check this link out for research
# https://stats.stackexchange.com/questions/199605/how-does-the-reparameterization-trick-for-vaes-work-and-why-is-it-important
def sampling(args):
    z_mean, z_log_sigma = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon 
    #      z_mean + e^(.5 * z_log_sigma)     * ϵ
    #                      φ

## Build the model

In [9]:
# build encoder model
encoder_input = Input(shape=(12,), name='Encoder_Input')

encoder_dense1 = Dense(512, activation='relu', name='Encoder_Dense1')(encoder_input)
encoder_dense2 = Dense(256, activation='relu', name='Encoder_Dense2')(encoder_dense1)
encoder_dense3 = Dense(128, activation='relu', name='Encoder_Dense3')(encoder_dense2)

# we need 2 Latent Sized Dense Layers for reparameterization
z_mean = Dense(2, name='z_mean')(encoder_dense3)
z_log_var = Dense(2, name='z_log_var')(encoder_dense3)

# Use Lambda layer to apply the sampling function (Reparameterization)
z = Lambda(sampling, output_shape=(2,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(encoder_input, [z_mean, z_log_var, z], name='Encoder')
encoder.summary()
# plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

Model: "Encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder_Input (InputLayer)      (None, 12)           0                                            
__________________________________________________________________________________________________
Encoder_Dense1 (Dense)          (None, 512)          6656        Encoder_Input[0][0]              
__________________________________________________________________________________________________
Encoder_Dense2 (Dense)          (None, 256)          131328      Encoder_Dense1[0][0]             
__________________________________________________________________________________________________
Encoder_Dense3 (Dense)          (None, 128)          32896       Encoder_Dense2[0][0]             
____________________________________________________________________________________________

In [10]:
# build decoder model
latent_inputs = Input(shape=(2,), name='z_sampling')

decoder_dense1 = Dense(128, activation='relu', name='Decoder_Dense1')(latent_inputs)
decoder_dense2 = Dense(256, activation='relu', name='Decoder_Dense2')(decoder_dense1)
decoder_dense3 = Dense(512, activation='relu', name='Decoder_Dense3')(decoder_dense2)

decoder_output = Dense(12, name='Decoder_Output')(decoder_dense3)

# instantiate decoder model
decoder = Model(latent_inputs, decoder_output, name='Decoder')
decoder.summary()
# plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

Model: "Decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 2)                 0         
_________________________________________________________________
Decoder_Dense1 (Dense)       (None, 128)               384       
_________________________________________________________________
Decoder_Dense2 (Dense)       (None, 256)               33024     
_________________________________________________________________
Decoder_Dense3 (Dense)       (None, 512)               131584    
_________________________________________________________________
Decoder_Output (Dense)       (None, 12)                6156      
Total params: 171,148
Trainable params: 171,148
Non-trainable params: 0
_________________________________________________________________


In [11]:
# instantiate VAE model
outputs = decoder(encoder(encoder_input)[2])
vae = Model(encoder_input, outputs, name='VAE_Model')

In [12]:
# Reconstuction Loss Function
reconstruction_loss = mse(encoder_input, outputs)
reconstruction_loss *= 12
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
vae.summary()
# plot_model(vae,
#             to_file='vae_mlp.png',
#             show_shapes=True)

Model: "VAE_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Encoder_Input (InputLayer)   (None, 12)                0         
_________________________________________________________________
Encoder (Model)              [(None, 2), (None, 2), (N 171396    
_________________________________________________________________
Decoder (Model)              (None, 12)                171148    
Total params: 342,544
Trainable params: 342,544
Non-trainable params: 0
_________________________________________________________________


In [14]:
vae.fit(x_train,
        epochs=20,
        batch_size=32)

Epoch 1/20
49985/49985 [==============================] - 7s 136us/step - loss: 8.6708
Epoch 2/20
49985/49985 [==============================] - 7s 135us/step - loss: 8.6318
Epoch 3/20
49985/49985 [==============================] - 7s 130us/step - loss: 8.5694
Epoch 4/20
49985/49985 [==============================] - 7s 130us/step - loss: 8.5559
Epoch 5/20
49985/49985 [==============================] - 7s 131us/step - loss: 8.5217
Epoch 6/20
49985/49985 [==============================] - 6s 130us/step - loss: 8.4926
Epoch 7/20
49985/49985 [==============================] - 6s 130us/step - loss: 8.4854
Epoch 8/20
49985/49985 [==============================] - 7s 132us/step - loss: 8.4570
Epoch 9/20
49985/49985 [==============================] - 7s 132us/step - loss: 8.4636
Epoch 10/20
49985/49985 [==============================] - 7s 132us/step - loss: 8.4522
Epoch 11/20
49985/49985 [==============================] - 7s 131us/step - loss: 8.4476
Epoch 12/20
49985/49985 [================

In [15]:
dump(encoder, open('VAE_Encoder.pkl', 'wb'))

# Old Method (Can't be Pickled)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import pprint
from pickle import dump




# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
# from keras.utils import plot_model

In [ ]:
songs = pd.read_csv("song_list5.csv")

In [ ]:
songs.head()

In [ ]:
songs.shape

In [ ]:
features = songs[[
    "danceability", "energy", "key", "loudness", "mode", "speechiness", 
    "acousticness", "instrumentalness", "liveness", "valence", "tempo", 
    "duration_ms"]].to_numpy()

In [ ]:
features[0]

In [ ]:
scaler = StandardScaler()

In [ ]:
x_train = scaler.fit_transform(features)

In [ ]:
dump(scaler, open('scaler2.pkl', 'wb'))

In [ ]:
print(x_train[0])
print(x_train[0].shape)

In [ ]:
encoder_input = Input(shape=(12,))

encoder_dense1 = Dense(512, activation='relu')(encoder_input)
encoder_dense2 = Dense(256, activation='relu')(encoder_dense1)
encoder_dense3 = Dense(128, activation='relu')(encoder_dense2)

encoder_output = Dense(2)(encoder_dense3)
encoder = Model(encoder_input, encoder_output, name='Encoder')
# encoder.compile()
encoder.summary()
# plot_model(encoder, to_file='encoder2.png', show_shapes=True)

In [ ]:
# print(encoder.predict(np.atleast_2d(x_train[0])))
# print(encoder.predict(np.atleast_2d(x_train[0])).shape)

In [ ]:
decoder_input = Input(shape=(2,))

decoder_dense1 = Dense(128, activation='relu')(decoder_input)
decoder_dense2 = Dense(256, activation='relu')(decoder_dense1)
decoder_dense3 = Dense(512, activation='relu')(decoder_dense2)

decoder_output = Dense(12)(decoder_dense3)
# decoder_output = Dense(13, activation='softplus')(decoder_dense5)

decoder = Model(decoder_input, decoder_output, name='Decoder')
decoder.summary()
# plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

In [ ]:
outputs = decoder(encoder(encoder_input))
auto_encoder = Model(encoder_input, outputs, name='Auto Encoder')



# auto_encoder = Model(encoder_input, decoder_output, name='Auto Encoder')

# auto_encoder = keras.Model(encoder_input, decoder_output)

auto_encoder.summary()

In [ ]:
auto_encoder.compile(optimizer='nadam',
                    loss='mean_absolute_error',
                    metrics='mean_absolute_percentage_error')

In [ ]:
x_train.shape

In [ ]:
type(auto_encoder)

In [ ]:
auto_encoder.fit(x_train, x_train,
                epochs=5,
                batch_size=32,
                shuffle=True)

In [ ]:
preds = auto_encoder.predict(x_train)

In [ ]:
pprint.pprint(preds[0])
pprint.pprint(x_train[0])
print("=======================================================================")
pprint.pprint(preds[1])
pprint.pprint(x_train[1])
print("=======================================================================")
pprint.pprint(preds[2])
pprint.pprint(x_train[2])

In [ ]:
preds2 = encoder.predict(x_train)

In [ ]:
pprint.pprint(preds2[0:19])

In [ ]:
encoder.save('encoder2.h5')

In [ ]:
# def __getstate__(self):
#     state = self.__dict__.copy()
#     del state['_metrics_lock']
#     return state

In [ ]:
# import threading

In [ ]:
# def __setstate__(self, state):
#     self.__dict__.update(state)
#     self.lock = threading.Lock() # ???

In [ ]:
# encoder.__dict__.copy()

In [ ]:
# __setstate__(encoder, __getstate__(encoder))


In [ ]:
dump(encoder, open('encoder2.pkl', 'wb'))